<a href="https://colab.research.google.com/github/IdjiotSandwiches/face-emotion-recognition/blob/knn-model/machine-learning/knn-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dagshub --quiet
!pip install mlflow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.5/252.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/

In [1]:
import cv2 as cv
import numpy as np
import mlflow
import dagshub
import os
import pathlib
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
PATH = pathlib.Path('C:\\Users\\vinar\\Downloads\\FER2013-augmented')
LABELS = os.listdir(PATH)

In [3]:
dagshub.init(repo_owner='IdjiotSandwiches', repo_name='face-emotion-recognition', mlflow=True)

Accessing as IdjiotSandwiches

Initialized MLflow to track repo "IdjiotSandwiches/face-emotion-recognition"

Repository IdjiotSandwiches/face-emotion-recognition initialized!

In [9]:
K_SIZE = (21,21)
SIGMA = 3
THETA_RANGE = np.arange(0, np.pi, np.pi/32)
LAMBD = 10.0
GAMMA = 0.5
PSI = 0
FLOATING_POINT = cv.CV_32F
IMAGE_SIZE = (48,48)
N_COMPONENTS = 0.95
N_COMPONENTS_LOCAL = 16
BLUR = (5,5)

gabor_params = {
    'ksize': K_SIZE,
    'sigma': SIGMA,
    'lambd': LAMBD,
    'gamma': GAMMA,
    'psi': PSI
}

KERNELS = [cv.getGaborKernel(**gabor_params, theta=theta) for theta in THETA_RANGE]

In [5]:
def gabor_filter(img):
  img = img.astype(np.float32)
  return np.array([cv.filter2D(img, FLOATING_POINT, kernel) for kernel in KERNELS])

In [6]:
def save_filtered_img(images, labels, path):
  folder_path = f'C:\\Users\\vinar\\Downloads\\gabor-filtered-imgs\\{path}'
  os.makedirs(folder_path, exist_ok=True)

  np.save(f'{folder_path}/images.npy', images)
  np.save(f'{folder_path}/labels.npy', labels)

  print('Ok!')

In [11]:
from joblib import Parallel, delayed

def process_image(img_path, label, dir):
    """
    Process a single image: read, preprocess, and apply PCA.
    """
    path = f'{dir}/{label}'
    img = cv.imread(f'{path}/{img_path}', 0)
    img = cv.resize(img, IMAGE_SIZE)
    img = cv.GaussianBlur(img, BLUR, 0)
    img = cv.equalizeHist(img)

    face_cascade = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))

    if len(faces) > 0:
        x, y, w, h = faces[0]
        img = img[y:y+h, x:x+w]
    else:
        h, w = img.shape[:2]
        crop_size = min(h, w)
        x = (w - crop_size) // 2
        y = (h - crop_size) // 2
        img = img[y:y+crop_size, x:x+crop_size]
    
    img = cv.copyMakeBorder(
        img, 
        10, 10, 10, 10,
        cv.BORDER_CONSTANT, 
        value=(0, 0, 0)
    )
    
    img = cv.resize(img, IMAGE_SIZE)
    img = img / 255.0
    img = gabor_filter(img)

    img = img.reshape(img.shape[0], -1)
    pca = PCA(n_components=N_COMPONENTS_LOCAL)
    img = pca.fit_transform(img)

    folder_path = f'C:\\Users\\vinar\\Downloads\\gabor-filtered-imgs\\FER2013\\new\\{label}'
    os.makedirs(folder_path, exist_ok=True)
    img_path = img_path.split('.')
    np.save(f'{folder_path}/{img_path[0]}.npy', img)


def open_dataset(dir):
    """
    Open dataset, load images, preprocess, and return images and labels using parallel processing.
    """
    # Use Parallel from joblib to process images in parallel
    results = Parallel(n_jobs=4)(delayed(process_image)(img_path, label, dir)
                                   for label in LABELS
                                   for img_path in tqdm(os.listdir(f'{dir}/{label}')))

    # Unzip the results into images and labels
    # images, labels = zip(*results)

    # return np.array(images), np.array(labels)

In [12]:
images, labels = open_dataset(PATH)
# save_filtered_img(images, labels, 'FER2013/new')

100%|██████████| 12000/12000 [02:10<00:00, 91.81it/s] 


TypeError: cannot unpack non-iterable NoneType object

In [13]:
images = []
labels = []
folder = 'C:\\Users\\vinar\\Downloads\\gabor-filtered-imgs\\FER2013\\new'
LABELS = os.listdir(folder)
for label in LABELS:
    class_folder = f'{folder}/{label}'
    for img_path in tqdm(os.listdir(class_folder)):
        img = np.load(f'{class_folder}/{img_path}')
        images.append(img)
        labels.append(label)

100%|██████████| 12000/12000 [01:16<00:00, 157.81it/s]


In [14]:
images = np.array(images)
labels = np.array(labels)

In [15]:
images = images.reshape(images.shape[0], -1)
pca = PCA(n_components=N_COMPONENTS)
images_ = pca.fit_transform(images)

print("Explained variance ratio:", pca.explained_variance_ratio_)
print("Cumulative explained variance:", np.cumsum(pca.explained_variance_ratio_))
print(images_.shape)

Explained variance ratio: [0.47909495 0.29831967 0.07858441 0.06036508 0.02089529 0.01489168]
Cumulative explained variance: [0.47909495 0.7774146  0.85599905 0.91636413 0.93725944 0.9521511 ]
(72000, 6)


In [17]:
save_filtered_img(images_, labels, 'FER2013/combined')

Ok!


In [18]:
test_images, test_labels = open_dataset(f'{PATH}/test')
save_filtered_img(test_images, test_labels, 'test')

Ok!


## **Testing**

In [7]:
images = np.load('C:\\Users\\vinar\\Downloads\\gabor-filtered-imgs\\FER2013\\combined\\images.npy')
labels = np.load('C:\\Users\\vinar\\Downloads\\gabor-filtered-imgs\\FER2013\\combined\\labels.npy')

N_NEIGHBORS = 9
WEIGHTS = 'distance'
METRIC = 'manhattan'
ALGORITHM = 'auto'
TEST_SIZE = 0.8
RANDOM_STATE = 42
N_SPLITS = 15
SHUFFLE = True

model_params = {
  'n_neighbors': N_NEIGHBORS,
  'weights': WEIGHTS,
  'metric': METRIC,
  'algorithm': ALGORITHM
}

calib_params = {
  'test_size': TEST_SIZE,
  'random_state': RANDOM_STATE
}

kfold_params = {
    'n_splits': N_SPLITS,
    'shuffle': SHUFFLE
}

skf = StratifiedKFold(**kfold_params)
i = 0

for train_idx, test_idx in skf.split(images, labels):
  X_train, X_test = images[train_idx], images[test_idx]
  y_train, y_test = labels[train_idx], labels[test_idx]

  X_calib, X_test, y_calib, y_test = train_test_split(X_test, y_test, **calib_params)

  model = KNeighborsClassifier(**model_params)
  model.fit(X_train, y_train)

  calib_model = CalibratedClassifierCV(model, cv="prefit")
  calib_model.fit(X_calib, y_calib)

  predict = calib_model.predict(X_test)
  predict_proba = calib_model.predict_proba(X_test)

  metrics = {
    'accuracy': accuracy_score(y_test, predict),
    'precision': precision_score(y_test, predict, average='macro'),
    'recall': recall_score(y_test, predict, average='macro'),
    'f1': f1_score(y_test, predict, average='macro'),
    'auc_score': roc_auc_score(y_test, predict_proba, multi_class='ovr', average='macro')
  }

  print(f"Accuracy: {metrics['accuracy']}\nPrecision: {metrics['precision']}\nRecall: {metrics['recall']}\nF1 Score: {metrics['f1']}\nAUC Score: {metrics['auc_score']}")
  report = classification_report(y_test, predict)
  print(report)

  mlflow.set_experiment("KNN_Model FER2013 augmented")
  mlflow.set_tracking_uri("https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow")

  with mlflow.start_run(run_name=f'KNN StratifiedKFold={i}'):
    mlflow.log_params(gabor_params)
    mlflow.log_params(calib_params)
    mlflow.log_params(model_params)
    mlflow.log_params(kfold_params)
    mlflow.log_param('floating_point', FLOATING_POINT)
    mlflow.log_param('image_size', IMAGE_SIZE)
    mlflow.log_param('PCA_n_components', N_COMPONENTS)
    mlflow.log_param('gaussian_blur', BLUR)
    mlflow.log_metrics(metrics)
    mlflow.sklearn.log_model(
        sk_model=calib_model,
        artifact_path='KNN Model',
        input_example=X_train[:1]
    )
  i = i + 1

Accuracy: 0.40302267002518893
Precision: 0.41299608247524017
Recall: 0.4048097415362125
F1 Score: 0.4061926739713124
AUC Score: 0.7423581160105994
              precision    recall  f1-score   support

       angry       0.43      0.34      0.38       661
     disgust       0.55      0.46      0.50       535
        fear       0.40      0.37      0.39       634
       happy       0.36      0.38      0.37       633
     neutral       0.32      0.42      0.36       621
         sad       0.41      0.39      0.40       652
    surprise       0.41      0.47      0.44       631

    accuracy                           0.40      4367
   macro avg       0.41      0.40      0.41      4367
weighted avg       0.41      0.40      0.40      4367



2024/12/16 20:54:13 INFO mlflow.tracking.fluent: Experiment with name 'KNN_Model FER2013 augmented' does not exist. Creating a new experiment.


🏃 View run KNN StratifiedKFold=0 at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/18/runs/e7ba29bcac2a4fe386bc657e01781808
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/18
Accuracy: 0.393405083581406
Precision: 0.4019841855378405
Recall: 0.39641527564110923
F1 Score: 0.39701216058685024
AUC Score: 0.7386236439677767
              precision    recall  f1-score   support

       angry       0.42      0.32      0.36       661
     disgust       0.54      0.50      0.52       535
        fear       0.35      0.39      0.37       634
       happy       0.38      0.38      0.38       633
     neutral       0.32      0.39      0.35       621
         sad       0.40      0.35      0.37       652
    surprise       0.41      0.45      0.43       631

    accuracy                           0.39      4367
   macro avg       0.40      0.40      0.40      4367
weighted avg       0.40      0.39      0.39

🏃 View run KNN StratifiedKFold=1 at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/18/runs/e683b158690b4008aa06cd0d68f73829
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/18
Accuracy: 0.3847034577513167
Precision: 0.39281919756271383
Recall: 0.3878716270835458
F1 Score: 0.3875802950607442
AUC Score: 0.7348155978680045
              precision    recall  f1-score   support

       angry       0.39      0.31      0.34       661
     disgust       0.53      0.49      0.51       535
        fear       0.41      0.34      0.37       634
       happy       0.33      0.35      0.34       633
     neutral       0.31      0.41      0.35       621
         sad       0.37      0.33      0.35       652
    surprise       0.42      0.49      0.45       631

    accuracy                           0.38      4367
   macro avg       0.39      0.39      0.39      4367
weighted avg       0.39      0.38      0.38

🏃 View run KNN StratifiedKFold=2 at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/18/runs/a71fb4ec80224eae9b0cfa129d24baaf
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/18
Accuracy: 0.39867185711014425
Precision: 0.40590718947518434
Recall: 0.4025302870846642
F1 Score: 0.401864305810431
AUC Score: 0.7387189929033056
              precision    recall  f1-score   support

       angry       0.40      0.31      0.35       661
     disgust       0.57      0.53      0.55       535
        fear       0.36      0.41      0.38       634
       happy       0.36      0.33      0.34       633
     neutral       0.34      0.43      0.38       621
         sad       0.39      0.34      0.36       652
    surprise       0.42      0.46      0.44       631

    accuracy                           0.40      4367
   macro avg       0.41      0.40      0.40      4367
weighted avg       0.40      0.40      0.40

...

🏃 View run KNN StratifiedKFold=10 at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/18/runs/6ea61f3e862a4fc5bbdadbac679f732d
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/18
Accuracy: 0.3985341273476867
Precision: 0.4055017723886456
Recall: 0.40106092945873345
F1 Score: 0.401009115259094
AUC Score: 0.7437075622028809
              precision    recall  f1-score   support

       angry       0.41      0.31      0.35       661
     disgust       0.52      0.48      0.50       534
        fear       0.36      0.40      0.38       634
       happy       0.36      0.38      0.37       634
     neutral       0.33      0.40      0.37       620
         sad       0.42      0.36      0.39       653
    surprise       0.44      0.47      0.46       630

    accuracy                           0.40      4366
   macro avg       0.41      0.40      0.40      4366
weighted avg       0.40      0.40      0.40

🏃 View run KNN StratifiedKFold=11 at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/18/runs/4077c3628d9148d392d1ed14a9b11fd4
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/18
Accuracy: 0.3783783783783784
Precision: 0.3867571236213267
Recall: 0.3813648402154365
F1 Score: 0.38154419822246727
AUC Score: 0.7341728352295253
              precision    recall  f1-score   support

       angry       0.39      0.30      0.34       661
     disgust       0.50      0.48      0.49       534
        fear       0.35      0.37      0.36       634
       happy       0.33      0.38      0.36       634
     neutral       0.29      0.37      0.33       620
         sad       0.42      0.34      0.37       653
    surprise       0.43      0.44      0.43       630

    accuracy                           0.38      4366
   macro avg       0.39      0.38      0.38      4366
weighted avg       0.38      0.38      0.3

🏃 View run KNN StratifiedKFold=12 at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/18/runs/6cbe6fc47e1c47c098d81a952acd6a03
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/18
Accuracy: 0.3760879523591388
Precision: 0.3817634271391193
Recall: 0.3784958694635025
F1 Score: 0.3788903689228461
AUC Score: 0.7323726168157021
              precision    recall  f1-score   support

       angry       0.38      0.32      0.35       661
     disgust       0.49      0.46      0.47       534
        fear       0.35      0.38      0.36       634
       happy       0.36      0.37      0.37       634
     neutral       0.31      0.38      0.34       620
         sad       0.36      0.32      0.34       653
    surprise       0.41      0.42      0.42       630

    accuracy                           0.38      4366
   macro avg       0.38      0.38      0.38      4366
weighted avg       0.38      0.38      0.38

🏃 View run KNN StratifiedKFold=13 at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/18/runs/fe453ce54dad45cabc3154d638cc2465
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/18
Accuracy: 0.3868529546495648
Precision: 0.3940975403116174
Recall: 0.3897644075151687
F1 Score: 0.389558539467925
AUC Score: 0.7338623819623314
              precision    recall  f1-score   support

       angry       0.39      0.31      0.34       661
     disgust       0.54      0.48      0.50       534
        fear       0.37      0.40      0.38       634
       happy       0.35      0.37      0.36       634
     neutral       0.33      0.39      0.36       620
         sad       0.39      0.32      0.35       653
    surprise       0.40      0.47      0.43       630

    accuracy                           0.39      4366
   macro avg       0.39      0.39      0.39      4366
weighted avg       0.39      0.39      0.39 

🏃 View run KNN StratifiedKFold=14 at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/18/runs/30e5ac1368194bdfb97badbf86c3f760
🧪 View experiment at: https://dagshub.com/IdjiotSandwiches/face-emotion-recognition.mlflow/#/experiments/18


## **Testing images shape**

In [25]:
images_ = []
# img = cv.imread(f'facial-emotion-recognition-augmented/disgust/disgust_1022.png', cv.IMREAD_GRAYSCALE)
# img = cv.imread(f'C:\\Users\\vinar\\Downloads\\RAF-DB\\test\\fear\\test_2253_aligned.jpg', cv.IMREAD_GRAYSCALE)
img = cv.imread(f'../neutral_3.png', cv.IMREAD_GRAYSCALE)
img = cv.resize(img, (100,100))
img = cv.GaussianBlur(img,(5,5),0)
img = cv.equalizeHist(img)
img = img / 255.0
img = gabor_filter(img)
img = np.array(img)
img = img.reshape(img.shape[0],-1)
pca = PCA(n_components=10)
img = pca.fit_transform(img)
img = img.reshape(-1)
images_.append(img)
images_ = np.array(images_)

print("Explained variance ratio:", pca.explained_variance_ratio_)
print("Cumulative explained variance:", np.cumsum(pca.explained_variance_ratio_))

print(images_.shape)

Explained variance ratio: [0.51456678 0.21078003 0.15307621 0.05976604 0.0324442  0.01573344
 0.0074042  0.00312239 0.00156306 0.00067005]
Cumulative explained variance: [0.51456678 0.72534682 0.87842302 0.93818907 0.97063326 0.9863667
 0.9937709  0.9968933  0.99845636 0.99912641]
(1, 1280)


In [8]:
logged_model = 'runs:/14b06e579a264caab691033399c45ceb/KNN Model'
model = mlflow.pyfunc.load_model(logged_model)

In [26]:
prediction = model.predict(images_)
for p in prediction:
  print(p)

happy


## **Tuning**

### **Without PCA**

In [10]:
images = np.load('C:\\Users\\vinar\\Downloads\\gabor-filtered-imgs\\FER2013\\new\\images.npy')
labels = np.load('C:\\Users\\vinar\\Downloads\\gabor-filtered-imgs\\FER2013\\new\\labels.npy')

N_NEIGHBORS = [3,5,7,9,11]
WEIGHTS = ['uniform', 'distance']
METRIC = ['minkowski', 'euclidean', 'manhattan']
ALGORITHM = ['auto', 'ball_tree', 'kd_tree', 'brute']

TEST_SIZE = 0.2
RANDOM_STATE = 42

split_params = {
    'test_size': TEST_SIZE,
    'random_state': RANDOM_STATE
}

model_params = {
    'n_neighbors': N_NEIGHBORS,
    'weights': WEIGHTS,
    'metric': METRIC,
    'algorithm': ALGORITHM
}

scoring = {
    'accuracy': 'accuracy',
    # 'roc_auc': 'roc_auc'
}

X_train, X_test, y_train, y_test = train_test_split(images, labels, **split_params, stratify=labels)
# X_test, X_calib, y_test, y_calib = train_test_split(X_test, y_test, **calib_params, stratify=y_test)

model = KNeighborsClassifier()
grid = GridSearchCV(
    estimator=model, 
    param_grid=model_params,
    scoring=scoring,
    refit='accuracy',
    cv=3,
    verbose=2
)
grid.fit(X_train, y_train)

print(grid.best_estimator_)
print(grid.best_params_)

Fitting 3 folds for each of 120 candidates, totalling 360 fits
[CV] END algorithm=auto, metric=minkowski, n_neighbors=3, weights=uniform; total time=   0.2s
[CV] END algorithm=auto, metric=minkowski, n_neighbors=3, weights=uniform; total time=   0.2s
[CV] END algorithm=auto, metric=minkowski, n_neighbors=3, weights=uniform; total time=   0.2s
[CV] END algorithm=auto, metric=minkowski, n_neighbors=3, weights=distance; total time=   0.0s
[CV] END algorithm=auto, metric=minkowski, n_neighbors=3, weights=distance; total time=   0.0s
[CV] END algorithm=auto, metric=minkowski, n_neighbors=3, weights=distance; total time=   0.0s
[CV] END algorithm=auto, metric=minkowski, n_neighbors=5, weights=uniform; total time=   0.2s
[CV] END algorithm=auto, metric=minkowski, n_neighbors=5, weights=uniform; total time=   0.2s
[CV] END algorithm=auto, metric=minkowski, n_neighbors=5, weights=uniform; total time=   0.2s
[CV] END algorithm=auto, metric=minkowski, n_neighbors=5, weights=distance; total time= 

KeyboardInterrupt: 

### **Using calibration**

In [5]:
images = np.load('C:\\Users\\vinar\\Downloads\\gabor-filtered-imgs\\FER2013\\new\\images.npy')
labels = np.load('C:\\Users\\vinar\\Downloads\\gabor-filtered-imgs\\FER2013\\new\\labels.npy')

N_NEIGHBORS = 9
WEIGHTS = 'distance'
METRIC = 'manhattan'
ALGORITHM = 'auto'

TEST_SIZE = 0.3
CALIB_SIZE = 0.5
RANDOM_STATE = 42

split_params = {
    'test_size': TEST_SIZE,
    'random_state': RANDOM_STATE
}

calib_params = {
    'test_size': CALIB_SIZE,
    'random_state': RANDOM_STATE
}

X_train, X_test, y_train, y_test = train_test_split(images, labels, **split_params, stratify=labels)
X_test, X_calib, y_test, y_calib = train_test_split(X_test, y_test, **calib_params, stratify=y_test)

model_params = {
  'n_neighbors': N_NEIGHBORS,
  'weights': WEIGHTS,
  'metric': METRIC,
  'algorithm': ALGORITHM
}

model = KNeighborsClassifier(**model_params)
model.fit(X_train, y_train)

calib_model = CalibratedClassifierCV(model, cv="prefit")
calib_model.fit(X_calib, y_calib)

predict = calib_model.predict(X_test)
predict_proba = calib_model.predict_proba(X_test)

metrics = {
  'accuracy': accuracy_score(y_test, predict),
  'precision': precision_score(y_test, predict, average='macro'),
  'recall': recall_score(y_test, predict, average='macro'),
  'f1': f1_score(y_test, predict, average='macro'),
  'auc_score': roc_auc_score(y_test, predict_proba, multi_class='ovr', average='macro')
}

print(f"Accuracy: {metrics['accuracy']}\nPrecision: {metrics['precision']}\nRecall: {metrics['recall']}\nF1 Score: {metrics['f1']}\nROC_AUC Score: {metrics['auc_score']}")
print(classification_report(y_test, predict))

# mlflow.set_experiment("KNN_Model using calibration")
# mlflow.set_tracking_uri("https://dagshub.com/IdjiotSandwiches/knn-fer.mlflow")

# with mlflow.start_run(run_name=f'KNN sigma=5'):
#   mlflow.log_params(gabor_params)
#   mlflow.log_params(split_params)
#   mlflow.log_param('calib_test_size', calib_params['test_size'])
#   mlflow.log_param('calib_random_state', calib_params['random_state'])
#   mlflow.log_params(model_params)
#   mlflow.log_metrics(metrics)
#   mlflow.sklearn.log_model(
#       sk_model=calib_model,
#       artifact_path='KNN Model',
#       input_example=X_train[:1]
#   )

Accuracy: 0.31209362808842656
Precision: 0.3955131143254627
Recall: 0.28298242490642067
F1 Score: 0.3025389865580613
ROC_AUC Score: 0.6350601746255912
              precision    recall  f1-score   support

       angry       0.32      0.15      0.21       743
     disgust       0.80      0.29      0.43        82
        fear       0.30      0.18      0.22       768
       happy       0.30      0.64      0.40      1348
     neutral       0.28      0.18      0.22       930
         sad       0.26      0.18      0.21       911
    surprise       0.51      0.36      0.42       601

    accuracy                           0.31      5383
   macro avg       0.40      0.28      0.30      5383
weighted avg       0.32      0.31      0.29      5383



### **Tuning k Value**

In [65]:
images = np.load('C:\\Users\\vinar\\Downloads\\gabor-filtered-imgs\\images.npy')
labels = np.load('C:\\Users\\vinar\\Downloads\\gabor-filtered-imgs\\labels.npy')

N_NEIGHBORS = range(1,21)
WEIGHTS = 'distance'
METRIC = 'manhattan'
ALGORITHM = 'ball_tree'
TEST_SIZE = 0.5
RANDOM_STATE = 42
N_SPLITS = 10
SHUFFLE = True

X_train, X_test, y_train, y_test = train_test_split(images, labels, random_state=RANDOM_STATE, test_size=0.2)

def evaluate_knn(k):
    model_params = {
        'n_neighbors': k,
        'weights': WEIGHTS,
        'metric': METRIC,
        'algorithm': ALGORITHM
    }
    model = KNeighborsClassifier(**model_params)
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    return cv_scores.mean()

# Parallel execution
scores = Parallel(n_jobs=4)(delayed(evaluate_knn)(k) for k in N_NEIGHBORS)

# Find optimal k
optimal_k = N_NEIGHBORS[scores.index(max(scores))]
print(f"Optimal k: {optimal_k}")

Optimal k: 1


## **Prediction result**

In [52]:
not_same = []

for p, y in zip(predict, y_test):
  if(p != y):
    not_same.append(p)

print(len(not_same))

1120
